# **Análise 01: Análise de Comportamento de Compra de Consumidores do E-commerce Brasileiro**
----

## **Introdução**

### Objetivo
A finalidade desta análise é examinar os dados dos consumidores do E-commerce brasileiro Olist, com o intuito de identificar os padrões de comportamento de compra e responder a diversas questões relacionadas ao negócio. Para alcançar esse objetivo, serão utilizadas diversas técnicas de análise de dados, incluindo a segmentação de clientes e análise de associação.


### Perguntas de Negócio
Está análise se propõe a responder as seguintes demandas:
* Quais são os produtos mais vendidos?
* Quais são os produtos que são geralmente comprados juntos?
* Qual é o valor médio de cada compra?
* Quais são os clientes mais valiosos?
* Qual é a taxa de recompra dos clientes?
* Quais são as preferências de compra dos clientes?
* Como as vendas variam ao longo do tempo?
* Como as vendas variam de acordo com a localização geográfica?
* Como o preço afeta o comportamento de compra dos clientes?
* Quais são as tendências de compra ao longo do tempo?

## Bibliotecas e Definições

Lista de todas as bibliotecas utilizadas, padronização de recursos e definições de funções.

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Etapa I - Entendimento dos dados e limpeza

Etapa de entendimento dos datasets utilizados, onde serão aplicados as técnicas de ETL (Extract, Transform and Load), bem como EDA (Exploratory Data Analysis)

### Coleta dos Dados
Carregamento dos dados que serão analisados

In [23]:
# Carregando dados dos Clientes
df_clientes = pd.read_csv('dados/olist_customers_dataset.csv', sep=',')
df_clientes.head(3)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP


In [25]:
# Carregando dados dos Pedidos
# utilizado argumento parse_dates para garantir que todas as colunas de datas estejam com o tipo datetime
df_pedidos = pd.read_csv('dados/olist_orders_dataset.csv', sep=',', parse_dates=['order_purchase_timestamp','order_approved_at', \
                                                                                 'order_delivered_carrier_date', 'order_delivered_customer_date', \
                                                                                 'order_estimated_delivery_date'])
df_pedidos.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04


In [29]:
# Carregandos dados dos Itens dos Pedidos
# utilizado argumento parse_dates para garantir que todas as colunas de datas estejam com o tipo datetime
df_itens_pedidos = pd.read_csv('dados/olist_order_items_dataset.csv', sep=',', parse_dates=['shipping_limit_date'])
df_itens_pedidos.head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87


### Entendendo os datasets
Utilizando técnicas de visualizações e resumos estátisticos para entender os dados e as variáveis de cada dataset

In [30]:
# Informações dos datasets

print(df_clientes.info(), '\n')
print(df_pedidos.info(), '\n')
print(df_itens_pedidos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_pur

## **Etapa II - Análise de Vendas**

## **Etapa III - Análise de Clientes**

## **Etapa IV - Análise geográfica e de preços**

## **Respostas Questionamentos**

## **Conclusão**

## **Referências**